In [1]:
print("hello, Darya!")

hello, Darya!


In [2]:
import numpy as np
import itertools as iter

In [3]:
def elements_row_xor(row1, row2, k):
    for i in range(k, len(row1)):
        row2[i] ^= row1[i]

In [4]:
def ref(matrix):
    k = -1
    for j in range(len(matrix[0])):
        b = True
        for i in range(k+1, len(matrix)):
            if matrix[i][j] == 1:
                if b:
                    k += 1
                    c = matrix[k]
                    matrix[k] = matrix[i]
                    matrix[i] = c
                    b = False
                else:
                    elements_row_xor(matrix[k], matrix[i], j)
    return matrix[:k+1]

In [5]:
def new_rref(matrix, k):
    x = 0
    col = 0
    while x < k:
        if matrix[x][col] == 1:
            for i in range(0, x):
                if matrix[i][col] == 1:
                    elements_row_xor(matrix[x], matrix[i], col)
            if col != x:
                _ = matrix[:, x].copy()
                matrix[:, x] = matrix[:, col].copy()
                matrix[:, col] = _
            x += 1
        col += 1
    return matrix

In [6]:
def createH(X, n, k):
    H = np.zeros((n, n-k), dtype=int)

    H[:k,:] = X.copy()

    H[k:,:] = np.eye(n-k,dtype=int)
    return H

In [7]:
def create_syndrome_table(H, err_count):
    n = len(H)
    one_err = np.eye(n, dtype=int)
    syndrome_table = {H[i].__str__(): one_err[i] for i in range(n)}

    for num in range(2, err_count + 1):
        syndrome_arr = [sum(arr) % 2 for arr in np.array(list(iter.combinations(H, num)))]
        err_arr = [sum(arr) for arr in np.array(list(iter.combinations(one_err, num)))]
        for i in range(len(syndrome_arr)):
            syndrome_table[syndrome_arr[i].__str__()] = err_arr[i]

    return syndrome_table

**Часть 1**

In [8]:
# начальная матрица
S = np.array([
    [1, 0, 0, 1, 0, 1, 1],
    [1, 1, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1, 1, 0]
])

**2.1. Сформировать порождающую матрицу линейного кода (7, 4, 3).**

In [9]:
G = ref(S.copy())
k = len(G)
n = len(G[0])

# приведённая матрица в ступенчатом виде
G = new_rref(G.copy(), k)
G

array([[1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1]])

In [10]:
n, k

(7, 4)

In [11]:
X = G[:,k:].copy()
X

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1]])

**2.2 Сформировать проверочную матрицу на основе порождающей.**

In [12]:
# проверочная матрица
H = createH(X.copy(), n, k)
H

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

**2.3 Сформировать таблицу синдромов для всех однократных ошибок.**

In [13]:
# сформируем таблицу синдромов для однократных ошибок
syndrome_table = create_syndrome_table(H.copy(), err_count=1)
syndrome_table

{'[1 1 1]': array([1, 0, 0, 0, 0, 0, 0]),
 '[1 1 0]': array([0, 1, 0, 0, 0, 0, 0]),
 '[1 0 1]': array([0, 0, 1, 0, 0, 0, 0]),
 '[0 1 1]': array([0, 0, 0, 1, 0, 0, 0]),
 '[1 0 0]': array([0, 0, 0, 0, 1, 0, 0]),
 '[0 1 0]': array([0, 0, 0, 0, 0, 1, 0]),
 '[0 0 1]': array([0, 0, 0, 0, 0, 0, 1])}

**2.4. Сформировать кодовое слово длины n из слова длины k.**

In [14]:
word = np.array([1, 0, 0, 1])
code_word = np.dot(word, G) % 2
code_word

array([1, 0, 0, 1, 1, 0, 0])

**Внести однократную ошибку в сформированное слово.**

In [15]:
err = np.array([0, 1, 0, 0, 0, 0, 0])
receive_code_word = code_word ^ err
receive_code_word

array([1, 1, 0, 1, 1, 0, 0])

**Вычислить синдром, исправить ошибку с использованием таблицы синдромов.**

In [16]:
syndrome = np.dot(receive_code_word, H) % 2
syndrome

array([1, 1, 0])

In [17]:
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
receive_code_word

array([1, 0, 0, 1, 1, 0, 0])

**Убедиться в правильности полученного слова.**

In [18]:
decoded_word = receive_code_word[:k]
decoded_word

array([1, 0, 0, 1])

In [19]:
np.array_equal(word, decoded_word)

True

**2.5. Сформировать кодовое слово длины n из слова длины k.**

In [20]:
word = np.array([0, 1, 1, 1])
code_word = np.dot(word, G) % 2
code_word

array([0, 1, 1, 1, 0, 0, 0])

**Внести двукратную ошибку в сформированное слово.**

In [21]:
err = np.array([0, 1, 0, 0, 0, 1, 0])
receive_code_word = code_word ^ err
receive_code_word

array([0, 0, 1, 1, 0, 1, 0])

**Вычислить синдром, исправить ошибку с использованием таблицы синдромов.**

In [22]:
syndrome = np.dot(receive_code_word, H) % 2
syndrome

array([1, 0, 0])

In [23]:
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
receive_code_word

array([0, 0, 1, 1, 1, 1, 0])

**Убедиться, что полученное слово отличается от отправленного.**

In [24]:
decoded_word = receive_code_word[:k]
decoded_word

array([0, 0, 1, 1])

In [25]:
np.array_equal(word, decoded_word)

False

**Часть 2**

**2.6. Сформировать порождающую матрицу линейного кода (n, k, 5).**

In [26]:
k, n = 5, 13
X = np.array([
    [1, 1, 1, 1, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 1, 1, 1, 1]
])
G = np.zeros((k,n), dtype=int)
G[:,:k] = np.eye(k, dtype=int)
G[:, k:] = X.copy()
G

array([[1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1]])

**2.7 Сформировать проверочную матрицу на основе порождающей.**

In [27]:
H = np.zeros((n, n-k), dtype=int)
H[:k, :] = X.copy()
H[k:, :] = np.eye(n-k, dtype=int)
H

array([[1, 1, 1, 1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

**2.8 Сформировать таблицу синдромов для всех однократных и двукратных ошибок.**

In [28]:
syndrome_table = create_syndrome_table(H.copy(), err_count=2)
syndrome_table

{'[1 1 1 1 1 0 0 1]': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 1 0 0 0 1 1 1]': array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 0 1 1 0 1 0 1]': array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 1 0 1 0 1 1]': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 0 0 1 1 1 1 1]': array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[1 0 0 0 0 0 0 0]': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 0 0 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 '[0 0 1 0 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 '[0 0 0 1 0 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
 '[0 0 0 0 1 0 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 '[0 0 0 0 0 1 0 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
 '[0 0 0 0 0 0 1 0]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 '[0 0 0 0 0 0 0 1]': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 '[0 0 1 1 1 1 1 0]': array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 '[0 1 0 0 1 1 0 0]'

**2.9. Сформировать кодовое слово длины n из слова длины k.Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов.**

In [29]:
word = np.array([0, 1, 1, 0, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word

array([0, 1, 1, 0, 1])

**Убедиться в правильности полученного слова.**

In [30]:
np.array_equal(word, decoded_word)

True

**2.10. Сформировать кодовое слово длины n из слова длины k. Внести двукратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов.**

In [31]:
word = np.array([1, 1, 0, 0, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word

array([1, 1, 0, 0, 1])

**Убедиться в правильности полученного слова.**

In [32]:
np.array_equal(word, decoded_word)

True

**2.1. Сформировать кодовое слово длины n из слова длины k. Внести трёхкратную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов.**

In [33]:
word = np.array([1, 1, 1, 1, 1])
code_word = np.dot(word, G) % 2
err = np.array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])
receive_code_word = code_word ^ err
syndrome = np.dot(receive_code_word, H) % 2
if not np.array_equal(syndrome, np.zeros(n, dtype=int)):
    receive_code_word ^= syndrome_table[syndrome.__str__()]
decoded_word = receive_code_word[:k]
decoded_word

array([0, 0, 1, 1, 0])

**Убедиться, что полученное слово отличается от отправленного.**

In [34]:
np.array_equal(word, decoded_word)

False